https://www.kaggle.com/code/pavansanagapati/ensemble-learning-techniques-tutorial

In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [3]:

iris = load_iris() 
X = iris.data[:, :4] 
Y = iris.target 

In [5]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state = 2) 

In [19]:
estimator = [] 
estimator.append(('LR',LogisticRegression(solver ='lbfgs',multi_class ='multinomial',max_iter = 200))) 
estimator.append(('SVC', SVC(gamma ='auto', probability = True))) 
estimator.append(('DTC', DecisionTreeClassifier())) 

In [32]:
hard_voting = VotingClassifier(estimators = estimator, voting ='soft') 
hard_voting.fit(X_train, y_train) 
y_pred = hard_voting.predict(X_test)   
# accuracy_score metric to predict Accuracy 
score = accuracy_score(y_test, y_pred) 
print("Hard Voting Score", score) 

Hard Voting Score 0.9666666666666667


In [33]:
y_test

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1,
       2, 1, 1, 0, 0, 2, 0, 2])

In [34]:
y_pred

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 2, 1,
       2, 1, 1, 0, 0, 2, 0, 2])

In [24]:
accuracy_score(y_test, y_pred) 

0.9666666666666667

In [30]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Assuming you have already loaded X and y from the Iris dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=2)

# Define the individual estimators
estimator_lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=200)
estimator_svc = SVC(gamma='auto', probability=True)
estimator_dtc = DecisionTreeClassifier()

# Train each individual estimator
estimator_lr.fit(X_train, y_train)
estimator_svc.fit(X_train, y_train)
estimator_dtc.fit(X_train, y_train)

# Make predictions for each estimator
y_pred_lr = estimator_lr.predict(X_test)
y_pred_svc = estimator_svc.predict(X_test)
y_pred_dtc = estimator_dtc.predict(X_test)

# Calculate accuracy for each estimator
score_lr = accuracy_score(y_test, y_pred_lr)
score_svc = accuracy_score(y_test, y_pred_svc)
score_dtc = accuracy_score(y_test, y_pred_dtc)

print("Logistic Regression Accuracy:", score_lr)
print("SVC Accuracy:", score_svc)
print("Decision Tree Accuracy:", score_dtc)

Logistic Regression Accuracy: 0.9666666666666667
SVC Accuracy: 0.9666666666666667
Decision Tree Accuracy: 0.9333333333333333


In [ ]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean